<a href="https://colab.research.google.com/github/jderazoa/ML-con-PYTHON/blob/master/recreating_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goals and progress
In this notebook we try to recreate Target calculation described in [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition/notebook#Building-your-prediction-model).

The code presented here is focused on proper Target calculation and is not optimized. The current notebook version provides close result, but not ideal. In average the difference is quite small, but for some rows gives values outside of normal Target range. I hope we could fix this with help of host, kaggle team and community.

**Update from 9-Nov-2021**

Thanks to investigation from Ernesto Budia (see his [notebook](https://www.kaggle.com/ebudia/recreating-target-min-periods-3750/)) we know how to get much closer match - when calculating $\beta^a$ rolling average $\langle .\rangle$ should be done for full 3750 minutes.

As a result of this change  
Average absolute error improved 0.00099 => 0.00006  
Max absolute error improved 2.44 => 0.045

**Update from 12-Nov-2021**

It turns out that naive approach we used at the beginning gives a lot of NA values for Recreated Target, so comparison between Target and Recreated target was done only for part of data (calculating Average/Max error does not reflect real picture). Additionally host published part of code, so we try to create faster version of current calculations using this code.

Average absolute error 0.000190  
Max absolute error 0.289100  
Standard deviation 0.001135  

**Update from 17 Nov-2021**

* use x-1 instead of log(x) when calculating return R - see [comment](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286778#1582764) by Branden Murray
* use unique timestamps (not all possible minutes as in previous version) - check line "all_timestamps = np.sort(data['timestamp'].unique())"

The last change means that all shift/rolling operations are done not in terms of minutes, but in terms of available records. The difference appears when a timestamp is missing for all assets.

Average absolute error 0.000190 => 0.000000  
Max absolute error 0.289100 => 0.003798  
Standard deviation 0.001135 = 0.000001  

# Naive approach
In this section we are implementing step by step calculation based on Target calculation description. It is slow and gives a lot of NA values in Recreated Target. So if you need faster/more correct version and not understanding of how it is calculated, jump to More optimized section.

Let's start with reading assets details, train data and adding Time column.

In [ ]:
import os
import numpy as np
import pandas as pd
import gc

directory = '../input/g-research-crypto-forecasting'
file_path = os.path.join(directory, 'train.csv')
dtypes = {
    'timestamp': np.int64,
    'Asset_ID': np.int8,
#     'Count': np.int32,
#     'Open': np.float64,
#     'High': np.float64,
#     'Low': np.float64,
    'Close': np.float64,
#     'Volume': np.float64,
#     'VWAP': np.float64,
    'Target': np.float64,
}
data = pd.read_csv(file_path, dtype=dtypes, usecols=list(dtypes.keys()))
data['Time'] = pd.to_datetime(data['timestamp'], unit='s')

file_path = os.path.join(directory, 'asset_details.csv')
details = pd.read_csv(file_path)

Then calculate return as per formula below. 

$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

This is done for each asset separately. We do not know which price should be used. There are five different prices: Open, High, Low, Close, and VWAP. Probably there is a mix like Open price for time + 1 min, and Close for time + 16 min. We use **Close** price in calculation below.

In [ ]:
price_column = 'Close'
ids = list(details.Asset_ID)
chunks = []
for id in ids:    
    asset = data[data.Asset_ID == id].copy()
    asset.sort_values(by='Time', inplace=True)
    asset.set_index(keys='Time', inplace=True)
    asset['p1'] = asset[price_column].shift(freq='-1T')
    asset['p16'] = asset[price_column].shift(freq='-16T')
    asset['r'] = np.log(asset.p16/asset.p1)
    asset.drop(['p1', 'p16'], axis=1, inplace=True)
    asset.reset_index(inplace=True)
    chunks.append(asset)

data = pd.concat(chunks)
data.sort_values(by='Time', inplace=True)

Next, assign weight for each row. And calculate M(t). Note that M(t) is the same for all assets and depend only on time.

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}$$

We do not know if ${\sum_a w^a}$ should be calculated for all assets or only for assets having data at time t.


In [ ]:
data['w'] = data['Asset_ID'].map(details.set_index(keys='Asset_ID')['Weight'])
weight_sum = details.Weight.sum()

data['weighted_asset_r'] = data.w * data.r
time_group = data.groupby('Time')

m = time_group['weighted_asset_r'].sum() / time_group['w'].sum()
#m = time_group['weighted_asset_r'].sum() / weight_sum

data.set_index(keys=['Time'], inplace=True)
data['m'] = m
data.reset_index(inplace=True)

After that, Beta is calculated. Bracket $\langle .\rangle$ represent the rolling average over time (3750 minute windows). If there is no full 3750 minute window, $\beta$ becomes zero.

$$\beta^a = \frac{\langle M \cdot R^a \rangle}{\langle M^2 \rangle}$$

In [ ]:
data['m2'] = data.m ** 2
data['mr'] = data.r * data.m

chunks = []
for id in ids:
    # type: pd.DataFrame
    asset = data[data.Asset_ID == id].copy()
    asset.sort_values(by='Time', inplace=True)
    asset.set_index(keys='Time', inplace=True)
    asset['mr_rolling'] = asset['mr'].rolling(window='3750T', min_periods=3750).mean()
    asset['m2_rolling'] = asset['m2'].rolling(window='3750T', min_periods=3750).mean()
    asset.reset_index(inplace=True)
    chunks.append(asset)
    debug = 1

data = pd.concat(chunks)
data.sort_values(by='Time', inplace=True)
data['beta'] = data['mr_rolling'] / data['m2_rolling']

And finallly Target is calculated.
$$\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

In [ ]:
data['Target_recreated'] = data['r'] - data['beta'] * data['m']

Now we compare given and recreated Target.

In [ ]:
data['Target_diff'] = np.abs(data['Target'] - data['Target_recreated'])

print(f'Average absolute error {data.Target_diff.mean():8.6f}')
print(f'Max absolute error {data.Target_diff.max():8.6f}')
print(f'Standard deviation {data.Target_diff.std():8.6f}')

Average absolute error 0.000062
Max absolute error 0.045587
Standard deviation 0.000189


Normal Target is changed in range [-0.5, 0.96], so Max absolute error 2.4 means recreated Target is completely wrong for some records.

In [ ]:
data['Target'].agg(['min', 'max'])

min   -0.509351
max    0.964170
Name: Target, dtype: float64

Let's check how many records with wrong (values outside of range) recreated Target do we have.

In [ ]:
(data.Target_recreated < -0.509351).sum()

0

In [ ]:
(data.Target_recreated > 0.96417).sum()

0

I have tried different prices, changed time intervals in formula, replaced rolling average from 3750 minutes to 3750 last records, etc. but result is the same or worse. It is possible minimize Max error, but avreage became higher.

Please let me know if you find an error in calculations or have some insight of how original Target is really calculated.

It turns out that this code gives a lot of NA values in Recreated Target, so minimizing Abs/Max error was perhaps misleading.

In [ ]:
pd.isna(data.Target).sum()

750338

In [ ]:
pd.isna(data.Target_recreated).sum()

9524636

Resulting Recreated Target has more than 10 times more NA values. Let's check when these NAs were intorduced.

In [ ]:
pd.isna(data.r).sum()

751424

When calculating R number of NAs is almost equal to number of Target NA. The difference is 1086. It can be explained by [missing first minite](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286095) of each month for each asset. 

It is interesting that the Target is present in train dataset for dates like YYYY-MM-01 23:44:00 (t+16) and YYYY-MM-01 23:59:00 (t+1). For these times calculation algorithm requires price at time YYYY-MM-01 00:00:00, but this time is absent in test data. It can be considered as a proof that Target was calculated on other data, and then train dataset was built with already calculated Target.

This missing first minute of a month bug gives approximately 44 month x 2 (every row affect two targets for shift 1 and 16) x 14 (number of assets) => 1232. Some assets do not have data for all months.

The only other place that can produce NA is the following code
> rolling(window='3750T', min_periods=3750)

missing values inside 3750 minutes interval result in NA values. 

Now let's look at the [code](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/286778) provided by host:
> num = df.multiply(mkt.values, axis=0).rolling(window).mean().values  
> denom = mkt.multiply(mkt.values, axis=0).rolling(window).mean().values  

There are two possible types of window that rolling accepts - [it can be](https://pandas.pydata.org/pandas-docs/version/1.3.0/reference/api/pandas.DataFrame.rolling.html) either shift/int (number or rows to shift) or time offset (number of minutes to shift), but there is no **min_periods** parameter. This means that for int shift it does not produce NA at all, for time offset it may produce NA only if there is no data for last 3750 minutes.

Next line of the code calculates beta and removes all resulting non-number values with zero:
> beta = np.nan_to_num( num.T / denom, nan=0., posinf=0., neginf=0.)  

The conclusion here is that adding min_periods=3750 which seemingly improved our metrics in fact added lots of NA values effectively minimizing data where these metrics were calculating. Host code shows that rolling average does not produce additional NA values.

# More optimized version

Here we try to do two things: solve problem with NA and make this code working faster, and re-use code from host.
The code from the host shows that data should be re-presented in a little bit different way to avoid expensive minutes-based shift/rolling operations and perform calculation for all assets at once.

We start with creating new data frame where all timestamps (minutes) present. In the train some minutes may be missing for some assets.

In [ ]:
ids = list(details.Asset_ID)
asset_names = list(details.Asset_Name)

# times = data['timestamp'].agg(['min', 'max']).to_dict()
# all_timestamps = np.arange(times['min'], times['max'] + 60, 60)
all_timestamps = np.sort(data['timestamp'].unique())
targets = pd.DataFrame(index=all_timestamps)


Next we calculate R for each asset and add its values as a column to new targets data frame. Note that some rows will contain NA as required prices with shift may be absent.

In [ ]:
for i, id in enumerate(ids):
    asset = data[data.Asset_ID == id].set_index(keys='timestamp')
    price = pd.Series(index=all_timestamps, data=asset[price_column])
#     targets[asset_names[i]] = np.log(
#         price.shift(periods=-16) /
#         price.shift(periods=-1)
#     )
    targets[asset_names[i]] = (
        price.shift(periods=-16) /
        price.shift(periods=-1)
    ) - 1



Next calculate M as weighted mean of columns (currently holding R value for each asset) for each row. This implementation count unavailable R values as zero. Please note it is not the only way to deal with NA values when calculating weighted average.

In [ ]:
weights = np.array(list(details.Weight))
targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)

Finally it is time to apply code provided by the host to calculate beta and Target. 

In [ ]:
m = targets['m']

num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

targets = targets - (beta * m.values).T

Let's check how it is different from given Target and how many NA values we have.

In [ ]:
diffs = []

for i, id in enumerate(ids):
    print(asset_names[i])
    # type: pd.DataFrame
    asset = data[data.Asset_ID == id].set_index(keys='timestamp')
    print(f'asset size {asset.shape[0]}')
    recreated = pd.Series(index=asset.index, data=targets[asset_names[i]])
    diff = np.abs(asset['Target'] - recreated)
    diffs.append(diff[~pd.isna(diff)].values)
    print(f'Average absolute error {diff.mean():8.6f}')
    print(f'Max absolute error {diff.max():8.6f}')
    print(f'Standard deviation {diff.std():8.6f}')
    print(f'Target na {pd.isna(asset.Target).sum()}')
    print(f'Target_calculated na {pd.isna(recreated).sum()}')
    print()

diffs = np.concatenate(diffs, axis=0)
print('For all assets')
print(f'Average absolute error {diffs.mean():8.6f}')
print(f'Max absolute error {diffs.max():8.6f}')
print(f'Standard deviation {diffs.std():8.6f}')


Bitcoin Cash
asset size 1953537
Average absolute error 0.000000
Max absolute error 0.003106
Standard deviation 0.000002
Target na 4861
Target_calculated na 4861

Binance Coin
asset size 1942619
Average absolute error 0.000000
Max absolute error 0.000000
Standard deviation 0.000000
Target na 13415
Target_calculated na 13415

Bitcoin
asset size 1956282
Average absolute error 0.000000
Max absolute error 0.003390
Standard deviation 0.000002
Target na 304
Target_calculated na 304

EOS.IO
asset size 1955140
Average absolute error 0.000000
Max absolute error 0.000000
Standard deviation 0.000000
Target na 2302
Target_calculated na 2302

Ethereum Classic
asset size 1951127
Average absolute error 0.000000
Max absolute error 0.000000
Standard deviation 0.000000
Target na 9326
Target_calculated na 9326

Ethereum
asset size 1956200
Average absolute error 0.000000
Max absolute error 0.003692
Standard deviation 0.000003
Target na 340
Target_calculated na 340

Litecoin
asset size 1956030
Average absol

It is still not ideal, but at least our metrics are calculated on almost all data available and number of NA almost the same as in Target.


There is a discussion about Beta equal to zero. It turns Target calculation just to R calculation. Let's see how many non-number values we have and how many rows have beta=0.

In [ ]:
beta_ = num.T / denom

for i, id in enumerate(ids):
    print(asset_names[i])
    print(f'Infiinte beta rows {np.isinf(beta_[i]).sum()}')
    nan_sum = np.isnan(beta_[i]).sum()
    print(f'NAN beta rows {nan_sum} ({100 * nan_sum / beta_.shape[1]:5.2f}%)')

    eps = 1e-6
    zero_sum = ((beta[i] > -eps) & (beta[i] < eps)).sum()
    print(f'Zero beta rows {zero_sum} ({100 * zero_sum / beta.shape[1]:5.2f}%)')
    print()

Bitcoin Cash
Infiinte beta rows 0
NAN beta rows 454219 (23.21%)
Zero beta rows 454219 (23.21%)

Binance Coin
Infiinte beta rows 0
NAN beta rows 1017774 (52.01%)
Zero beta rows 1017774 (52.01%)

Bitcoin
Infiinte beta rows 0
NAN beta rows 26172 ( 1.34%)
Zero beta rows 26172 ( 1.34%)

EOS.IO
Infiinte beta rows 0
NAN beta rows 197034 (10.07%)
Zero beta rows 197034 (10.07%)

Ethereum Classic
Infiinte beta rows 0
NAN beta rows 910686 (46.54%)
Zero beta rows 910686 (46.54%)

Ethereum
Infiinte beta rows 0
NAN beta rows 34042 ( 1.74%)
Zero beta rows 34042 ( 1.74%)

Litecoin
Infiinte beta rows 0
NAN beta rows 105904 ( 5.41%)
Zero beta rows 105904 ( 5.41%)

Monero
Infiinte beta rows 0
NAN beta rows 1662634 (84.97%)
Zero beta rows 1662634 (84.97%)

TRON
Infiinte beta rows 0
NAN beta rows 404944 (20.69%)
Zero beta rows 404944 (20.69%)

Stellar
Infiinte beta rows 0
NAN beta rows 936276 (47.85%)
Zero beta rows 936276 (47.85%)

Cardano
Infiinte beta rows 0
NAN beta rows 801939 (40.98%)
Zero beta rows 

Below is the same code but in the form of a function you can copy and paste. Note that it does not require Time column, it uses timestamp instead.

In [ ]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_Name)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = (
            price.shift(periods=-16) /
            price.shift(periods=-1)
        ) - 1
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)
    
    return targets